---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
df.tail(10)

490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

In [3]:
df[:15]

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
dtype: object

In [4]:
df[125:135]

125    s The patient is a 44 year old married Caucasi...
126    .10 Sep 2004 - Intake at EEC for IOP but did n...
127    see above and APS eval of 26 May 1982 Social H...
128    Tbooked for intake appointment at Sierra Vista...
129                    06 May 1972 SOS-10 Total Score:\n
130      25 Oct 1987 Total time of visit (in minutes):\n
131                    14 Oct 1996 SOS-10 Total Score:\n
132    30 Nov 2007 CPT Code: 90801 - Psychiatric Diag...
133    h missed intake office visit on 28 June 1994 a...
134                    14 Jan 1981 SOS-10 Total Score:\n
dtype: object

In [5]:
df[230:245]

230    B/R Walnut Ridge. Raised with sister and paren...
231    50 yo DWF with a history of alcohol use disord...
232    )HTN, hypercholesterolemia, DM, sleep apnea,, ...
233    Dr. Gloria English, who conducted an initial c...
234    . Patient presents with GAD flareup in Jul 200...
235    pOct 2015 - Admitted to Gray Clinic for depres...
236    Pt does not use marijuana currently. Her last ...
237    safter evicted in February 1976, hospitalized ...
238    . History of sleep apnea, set up with CPAP in ...
239    . In Feb 1978, misdxed with STD- had plan to O...
240    )- Venlafaxine 37.5mg daily: May, 2011: self-d...
241    sLanguage based learning disorder, dyslexia.  ...
242    oFather died in Nov 2010.  Normal grieving.Alc...
243    tIn Sep 2012 patient returned to Jinan, moving...
244    s Mr. Moss is a 27-year-old, Caucasian, engage...
dtype: object

In [6]:
df[485:]

485    Was "average" student.  "I didn't have too man...
486    Contemplating jumping off building - 1973 - di...
487    appendectomy s/p delivery 1992 Prior relevant ...
488    tProblems renal cell cancer : s/p nephrectomy ...
489        ran own business for 35 years, sold in 1985\n
490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

In [7]:
df[298]

'HH, Janaury 1993\n'

In [8]:
df = df.str.replace('Janaury', 'January')
df = df.str.replace('Decemeber', 'December' )

In [9]:
df[298]

'HH, January 1993\n'

In [10]:
import re

from dateutil.parser import parse
from datetime import datetime

In [11]:
df[346:383]

346                        9/2005 Primary Care Doctor:\n
347    s 03/1980 Positive PPD: treated with INH for 6...
348    12/2005 Family Psych History: Family History o...
349                        5/1987 Primary Care Doctor:\n
350                        5/2004 Primary Care Doctor:\n
351    A pleasant 28 yo woman with no formal psychiat...
352    See initial PROMPTCARE evaluation from 3/1986 ...
353                       10/1997 Primary Care Doctor:\n
354    Deviated septum, 3/1993 Activities of Daily Li...
355                        3/1981 Primary Care Doctor:\n
356    . Patient states that she will follow up with ...
357                           % 10/1993 Echo: LVEF 60%\n
358                              .Mother died, 1/1983.\n
359                                    . GBSG : 7/1994\n
360                       12/2008 Primary Care Doctor:\n
361    n Abnormal CXR : 10/1980 ? scarring @ R heart ...
362    md. metabolic montioring as indicated inc. ann...
363                       12/19

In [12]:
b = df.str.extract(r'((?:\d{1,2})(?:(?:\/|-)\d{1,2})(?:(?:\/|-)\d{2,4}))')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [13]:
a = df.str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [14]:
c = df.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{4})')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [15]:
a.notnull().sum()

218

In [16]:
b.notnull().sum()

125

In [17]:
c.notnull().sum()

428

In [18]:
p = a.fillna(b)
p = p.fillna(c)

In [19]:
from dateutil.parser import parse, parserinfo

class CustomParserInfo(parserinfo):
    def convertyear(self, year, *args, **kwargs):
        if year < 100:
            year += 1900
        return year

dflt_date = datetime(1993, 1, 1)
d = p.to_dict()
for ind, val in d.items():
    try:
        val = parse(val, default=dflt_date, dayfirst=False, parserinfo=CustomParserInfo())
    except ValueError:
        print (val)
        val = dflt_date
    d[ind] = val

In [20]:
d = sorted(d.items(), key=lambda x: x[1])

In [21]:
def date_sorter(): 
    df = pd.DataFrame(d)
    df.columns=['order', 'value']
       
    return df['order']

In [22]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: order, Length: 500, dtype: int64